# env

```bash
date
    Thu Apr 10 00:00:00 PM CEST 2025

cat /etc/os-release 
    PRETTY_NAME="Debian GNU/Linux 12 (bookworm)"
    NAME="Debian GNU/Linux"
    VERSION_ID="12"
    VERSION="12 (bookworm)"
    VERSION_CODENAME=bookworm
    ID=debian

python3 --version
    Python 3.11.2 
```

## google search results
- [logging — Logging facility for Python](https://docs.python.org/3.11/library/logging.html#module-logging)
- [python logging in jupyter notebooks](https://www.mineo.app/blog-page/python-logging-in-jupyter-notebooks)

## first simple example

<div class="alert alert-block alert-warning"> 
⚠️ We will see the DEBUG message is missing
</div>

In [ ]:
import logging

# logging.basicConfig(level=logging.INFO,  encoding='utf-8',format='%(asctime)s - %(levelname)s %(filename)s %(linenumber)s - %(message)s')
logging.basicConfig(level=logging.INFO,  encoding='utf-8',
                    format='%(asctime)s - %(levelname)s - %(message)s')

logging.debug("This is a debug message")
logging.info("This is an info message")
logging.warning("This is a warning message")
logging.error("This is an error message")
logging.critical("This is a critical message")

> [Sure, check formatters in logging docs. Specifically the lineno and pathname variables.](https://stackoverflow.com/questions/533048/how-to-log-source-file-name-and-line-number-in-python)

> - %(pathname)s Full pathname of the source file where the logging call was issued(if available).
> - %(filename)s Filename portion of pathname.
> - %(module)s Module (name portion of filename).
> - %(funcName)s Name of function containing the logging call.
> - %(lineno)d Source line number where the logging call was issued (if available).
> - Looks something like this:
>   ```python
>   formatter = logging.Formatter('[%(asctime)s] p%(process)s {%(pathname)s:%(lineno)d} %(levelname)s - %(message)s','%m-%d %H:%M:%S')
>   ```

# enable debug message 

<div class="alert alert-block alert-info">
📝  These both lines of code enable the logging.debug message

```python
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
```

</div>

## answer found here
  - [Discussions on Python.org: No debug messages going into log file](https://discuss.python.org/t/no-debug-messages-going-into-log-file/57897/15)

In [ ]:
import logging


logging.basicConfig(level=logging.DEBUG, encoding='utf-8',
                    format='%(asctime)s - %(levelname)s - %(message)s')

# [enable DEBUG level to log](https://discuss.python.org/t/no-debug-messages-going-into-log-file/57897/15)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


def main():
    logging.debug("This is a DEBUG message")
    logging.info("This is an INFO message")
    logging.warning("This is a WARNING message")
    logging.error("This is an ERROR message")
    logging.critical("This is a CRITICAL message")


main()

# logging from local file to local file

```bash
filename=python_logging_file_1.py

cat<<-EOF>$filename 
import logging
from pathlib import Path

# create log file name
log_file_name=Path(__file__).stem + ".log"

logger = logging.getLogger(__name__)

def main():
    logging.basicConfig(filename=log_file_name,
                        level=logging.DEBUG,
                        format="[%(asctime)s] p%(process)s {%(pathname)s:%(lineno)d} %(levelname)s - %(message)s")
    logging.debug("This is a debug message")
    logging.info("This is an info message")
    logging.warning("This is a warning message")
    logging.error("This is an error message")
    logging.critical("This is a critical message")

    logger.info('Started')
    logger.debug('working')
    logger.info('Finished')


if __name__ == '__main__':
    main()
EOF
```

# [customize logger format](https://stackoverflow.com/questions/1343227/can-pythons-logging-format-be-modified-depending-on-the-message-log-level)

# Custom formatter
class MyFormatter(logging.Formatter):

    err_fmt  = "ERROR: %(msg)s"
    dbg_fmt  = "DBG: %(module)s: %(lineno)d: %(msg)s"
    info_fmt = "%(msg)s"


    def __init__(self, fmt="%(levelno)s: %(msg)s"):
        logging.Formatter.__init__(self, fmt)


    def format(self, record):

        # Save the original format configured by the user
        # when the logger formatter was instantiated
        format_orig = self._fmt

        # Replace the original format with one customized by logging level
        if record.levelno == logging.DEBUG:
            self._fmt = MyFormatter.dbg_fmt

        elif record.levelno == logging.INFO:
            self._fmt = MyFormatter.info_fmt

        elif record.levelno == logging.ERROR:
            self._fmt = MyFormatter.err_fmt

        # Call the original formatter class to do the grunt work
        result = logging.Formatter.format(self, record)

        # Restore the original format configured by the user
        self._fmt = format_orig

        return result

In [ ]:
import logging
from accelerate import Accelerator
from accelerate.logging import get_logger
from rich.logging import RichHandler

accelerator = Accelerator()

my_logger = logging.getLogger(__name__)
my_logger.setLevel(logging.INFO)
formatter = logging.Formatter(
    fmt="%(name)s: %(lineno)s - %(message)s", datefmt="%m/%d %H:%M:%S")
handler = RichHandler(show_time=True, show_level=True, show_path=True)
handler.setFormatter(formatter)
my_logger.addHandler(handler)

logger = get_logger(__name__)
logger.logger = my_logger
logger.info("Test")